# hw4 - NN pyTorch

Во всем задании вы должны работать со входом из строк размером (длиной) 1024

## Часть 1 Фурье

In [21]:
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [102]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using mps device


### Подготовка данных

Сгенерите данные (сигнал) любым пакетом для численного преобразования Фурье, [например](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html#numpy.fft.fft)

In [92]:
class FourierDataset(Dataset):

    def __init__(self, size=500, sample_size=1024):
        self.size = size
        self.sample_size = sample_size
        self.samples, self.ffts = self.__generate_data()

    def __generate_data(self):
        samples = []
        ffts = []

        for i in range(self.size):
            samples.append(torch.from_numpy(np.random.normal(0, 1, size=self.sample_size)).type(torch.float32))
            ffts.append(torch.from_numpy(np.fft.fft(samples[-1]).real).type(torch.float32))

        return torch.stack(samples), torch.stack(ffts)

    def __len__(self):
        return self.size

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        return self.samples[index].squeeze(), self.ffts[index].squeeze()

In [94]:
dataset_length = 100000

train_dataset = FourierDataset(int(dataset_length * 0.8))
test_dataset = FourierDataset(int(dataset_length * 0.2))

### Соберите и обучите нейросетку на pyTorch для преобразования Фурье

In [96]:
class NeuralNetwork(nn.Module):

    def __init__(self, num_samples=1024, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linear1 = nn.Linear(num_samples, 4 * num_samples)
        self.linear2 = nn.Linear(4 * num_samples, 4 * num_samples)
        self.linear3 = nn.Linear(4 * num_samples, 4 * num_samples)
        self.linear4 = nn.Linear(4 * num_samples, num_samples)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        return self.linear4(x)

In [97]:
learning_rate = 1e-3

model = NeuralNetwork(1024).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [98]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    total_loss = 0

    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        if batch % 100 == 0:
            current = batch * len(X)
            print(f"Batch {batch}, loss: {loss.item():.6f} [{current:>5d}/{size:>5d}]")

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [99]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            loss = loss_fn(pred, y)
            total_loss += loss.item()

    avg_loss = total_loss / num_batches
    return avg_loss

In [100]:
train_dataloader = DataLoader(train_dataset)
test_dataloader = DataLoader(test_dataset)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t + 1}\n---------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    print("Done!")

Epoch 1
---------------------------
Batch 0, loss: 506.526489 [    0/80000]
Batch 100, loss: 520.855835 [  100/80000]
Batch 200, loss: 560.580322 [  200/80000]
Batch 300, loss: 506.946167 [  300/80000]
Batch 400, loss: 504.381104 [  400/80000]
Batch 500, loss: 493.187958 [  500/80000]


KeyboardInterrupt: 

### Сравните свое решение с пакетным методом

## Часть 2 Power spectral density (dB)

### Подготовка данных

Используя пакетное решение, [например](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html), подготовьте обучающую выборку.

### Соберите и обучите нейросеть для предсказания спектральной плотности мощности

*Подсказка: для входа 1024 выход будет 512*

### Провалидируйте свое решение

*Примечание: Для сдачи достаточно качественного соответствия по форме спектра с пакетным решением*